# Clean & Merge/Match 3 Datasets

In [1]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [29]:
df = pd.read_csv(r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\zoom_logs.csv")
df.shape

(277988, 23)

In [31]:
#Extract 5-digit SID from log in zoom name in case the student does not log in with verified school email
mask = (df['role'] == 'Student') & (df['SID_str'].isna())
extracted = (
    df.loc[mask, 'Name (original name)']
      .astype(str)
      .str.extract(r'(?<!\d)(\d{5}|\d{8})(?!\d)', expand=False)
)

# Assign only where we actually found a match
df.loc[mask, 'SID_str'] = extracted

#Check the newly extracted rows
df1 = df.loc[mask]
df1.head()

,Topic,ID,Host name,Host email,Start time,End time,Participants,Duration (minutes),Name (original name),Email,...,Guest,SID,date,time,day_of_week,hour,stu_join_time,stu_leave_time,SID_str,role
1,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,"Hoang Anh, Bao",NaN,...,Yes,NaN,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,NaN,Student
9,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,#Nguyen Tam Doan 04114 ( Nguyen Tam Doan 04114 ),NaN,...,Yes,NaN,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,04114,Student
13,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Thao Nguyen Nguyen,NaN,...,Yes,NaN,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,NaN,Student
19,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,"#Nguyen Hong Hanh Nhien 00790 ( ""Nguyen Nhien,...",NaN,...,Yes,NaN,9/30/2024,19:25:00,Monday,19,19:26:00,20:25:00,00790,Student
21,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,**Phạm Ngọc Kim Ngân 04410 ( Phạm Ngọc Kim Ngâ...,NaN,...,Yes,NaN,9/30/2024,19:25:00,Monday,19,19:26:00,20:25:00,04410,Student


In [33]:
#CHange SID column from float to str without 0s at the bback
df['SID'] = df['SID'].astype('Int64')
df['SID']

0             <NA>
1             <NA>
2         23001816
3         22002469
4         24002190
            ...   
277983    19050342
277984    19070116
277985    22002927
277986    19052826
277987    19052826
Name: SID, Length: 277988, dtype: Int64

In [34]:
# Identify rows where SID_str is exactly 8 digits (no letters or symbols)
mask_8 = df['SID_str'].astype(str).str.match(r'^\d{8}$', na=False)

# Transfer only those 8-digit numeric strings to SID
df.loc[mask_8, 'SID'] = df.loc[mask_8, 'SID_str']

# (Optional) Clear them from SID_str after moving
# df.loc[mask_8, 'SID_str'] = pd.NA


In [36]:
df['SID_5digit'] = df['SID_str'].astype(str).str.extract(r'(\d{5})$')
df.head()

,Topic,ID,Host name,Host email,Start time,End time,Participants,Duration (minutes),Name (original name),Email,...,SID,date,time,day_of_week,hour,stu_join_time,stu_leave_time,SID_str,role,SID_5digit
0,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,TA Phung ( schedule 1 ) (Host),schedule1@minhvietacademy.org,...,<NA>,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,schedule1,TA/Teacher,NaN
1,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,"Hoang Anh, Bao",NaN,...,<NA>,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,NaN,Student,NaN
2,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Mai Nguyen Truc Linh 01816,23001816@minhvietacademy.org,...,23001816,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,23001816,Student,01816
3,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Nguyen Pham Nhat Minh 02469,22002469@minhvietacademy.org,...,22002469,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,22002469,Student,02469
4,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Yen Ha Tu Vy 02190,24002190@minhvietacademy.org,...,24002190,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,24002190,Student,02190


#### Match Grade

In [37]:
#read the topic_class names dictionary to match the zoom classes to the right Grade
df_grade_class_dict = pd.read_excel(r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\goitenemmai\SY25_MVA_Goitenemmai (1).xlsx", sheet_name="grade_class_dict")

df_grade_class_dict

,Topic,Grade
0,2500COM,Grade K
1,2501HUM,Grade 1
2,2502HUM,Grade 2
3,2503HUM,Grade 3
4,2504HUM,Grade 4
5,2505HUM,Grade 5
6,2501MAT,Grade 1
7,2502MAT,Grade 2
8,2503MAT,Grade 3
9,2504MAT,Grade 4


In [38]:
df_merged = pd.merge(df, df_grade_class_dict, on="Topic", how="left")
df_merged.head()

,Topic,ID,Host name,Host email,Start time,End time,Participants,Duration (minutes),Name (original name),Email,...,date,time,day_of_week,hour,stu_join_time,stu_leave_time,SID_str,role,SID_5digit,Grade
0,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,TA Phung ( schedule 1 ) (Host),schedule1@minhvietacademy.org,...,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,schedule1,TA/Teacher,NaN,Grade 1
1,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,"Hoang Anh, Bao",NaN,...,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,NaN,Student,NaN,Grade 1
2,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Mai Nguyen Truc Linh 01816,23001816@minhvietacademy.org,...,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,23001816,Student,01816,Grade 1
3,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Nguyen Pham Nhat Minh 02469,22002469@minhvietacademy.org,...,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,22002469,Student,02469,Grade 1
4,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Yen Ha Tu Vy 02190,24002190@minhvietacademy.org,...,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,24002190,Student,02190,Grade 1


In [ ]:
df_merged.to_excel('extracted_SID.xlsx', index=False)

#### Match student list

In [ ]:
#read the list of students dataset
df_students = pd.read_excel(r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\goitenemmai\SY25_MVA_Goitenemmai (1).xlsx")
df_students

,SID,SID_8digit,SID_5digit,Name,Grade,Grade_bytopic,EFL,Note,MVA/MVSM,concat,4 last digits,Học sinh cùng lớp có chung 4 số cuối ID
0,2100-0802,21000802,802,Nguyễn Như Việt,PreK,PreK,EFL K5,Chính thức,MVA,MVA2100-0802,802.0,NaN
1,2200-0902,22000902,902,Mai Nguyễn Minh Châu,PreK,PreK,EFL K5,Chính thức,MVA,MVA2200-0902,902.0,NaN
2,2200-2125,22002125,2125,Vũ Lam Cát Tường,PreK,PreK,EFL K5,Chính thức,MVA,MVA2200-2125,2125.0,lưu ý trùng 4 số cuối ID
3,2200-2477,22002477,2477,Trần Thị Thục Quyên,PreK,PreK,EFL K5,Chính thức,MVA,MVA2200-2477,2477.0,NaN
4,2200-3681,22003681,3681,Nguyễn Linh San,PreK,PreK,EFL K5,Chính thức,MVA,MVA2200-3681,3681.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2240,2400-1675,24001675,1675,Lê Mỹ Anh,BA 1,BA 1,NaN,chính thức,MVSM,MVSM2400-1675,1675.0,NaN
2241,2400-1785,24001785,1785,Lê Vũ Minh,BA 1,BA 1,NaN,chính thức,MVSM,MVSM2400-1785,1785.0,NaN
2242,2400-1787,24001787,1787,Nguyễn Hải Trúc,BA 1,BA 1,NaN,chính thức,MVSM,MVSM2400-1787,1787.0,NaN
2243,2400-1880,24001880,1880,Hồ Thị Phương Anh,BA 1,BA 1,EFL K5,chính thức,MVSM,MVSM2400-1880,1880.0,NaN


### Merging

In [ ]:
#Find the list of students on zoom (df) that doesnt match with any in the official Student list
m = pd.merge(
    df_merged,
    df_students,
    left_on='SID',
    right_on='SID_8digit',
    how='outer',
    indicator=True
)
m.shape

(280233, 38)

#### Find the list of students that don't match

In [16]:
zoom_only = m[m['_merge'] == 'left_only']
zoom_only.shape
#zoom_only.to_excel('zoom_only.xlsx', index=False)

(277988, 38)

In [93]:
students_only = m[m['_merge']=='right_only']
students_only.shape

(713, 38)

In [92]:
not_matched = m[~(m['_merge'] == 'both')]
not_matched.shape

(50976, 38)

In [91]:
matched = m[m['_merge'] == 'both']
matched.shape

(227725, 38)